# 第四节 实战召回优化——大模型查询重写


上一节我们讲了召回的重要性，这一节内容我们将在代码层面实现优化召回率的其中一种方式——大模型查询重写。

## 提升指标的优化策略

做了这么久 RAG，我最大的感受就是——**没有完美的方案，都是在各种 trade-off 中找平衡**。不同的业务场景需要不同的优化重点，有时候甚至需要牺牲一些召回率来换取精确度。关键是要根据实际情况灵活调整，别死磕某一个指标。

**查询重写**这块儿，我之前碰到过一个特别典型的案例——用户问"这个怎么处理"，你说这让系统怎么理解？后来我们就在查询重写上下了功夫。比如**在医疗领域的项目**里，用户说"**头疼**"，我们会自动扩展成"**头痛、偏头痛、头部疼痛**"这些同义词，召回率一下子就上去了。不过说真的，这招在**专业领域特别好使，通用场景下反而容易引入噪音**。

那么如何从**代码层面实现大模型查询重写优化策略**呢？

## 基于大模型的查询重写策略

这种策略可以细分成**查询扩写**、**子问题查询**和**多步骤查询**三类。我们一点一点实现。

### 查询扩写

用户查询常常信息不足或存在歧义，影响检索效果。查询扩写通过同义词补充、上下文补全、模板转换等方式，使查询更清晰具体，从而提升准确率和召回率。我们还是基于 [cmrc2018](https://huggingface.co/datasets/LazyAGI/CMRC2018_Knowledge_Base/tree/main) 数据集做的 RAG 系统继续优化，上代码：

In [ ]:
pip install lazyllm
export QWEN_API_KEY='sk-这里填写你的key'

In [ ]:
# 查询扩写
# 导入必要的库
import lazyllm
import os
from lazyllm import Document, ChatPrompter, Retriever

# 查询改写提示词
# 用于将用户的查询改写得更加清晰，不回答问题
rewrite_prompt = "你是一个查询改写助手，将用户的查询改写的更加清晰。\
                注意，你不需要对问题进行回答，只需要对原始问题进行改写。\
                下面是一个简单的例子：\
                输入：RAG\
                输出：为我介绍下RAG。\
                    \
                用户输入为："

# AI问答助手提示词
# 用于根据上下文和问题提供答案
robot_prompt = "你是一个友好的 AI 问答助手，你需要根据给定的上下文和问题提供答案。\
                根据以下资料回答问题：\
                {context_str} \n"

# 加载文档库，定义检索器和在线大模型
# 创建文档对象，指定数据集路径
documents = Document(dataset_path="./data_kb")  # 请在 dataset_path 传入数据集绝对路径
# 创建检索器，指定文档、分组名、相似度算法和返回结果数量
retriever = Retriever(
    doc=documents, group_name="CoarseChunk", similarity="bm25_chinese", topk=3
)  # 定义检索组件
# 创建在线大模型对象，指定来源、模型和API密钥
llm = lazyllm.OnlineChatModule(
    source="qwen",
    model="qwen-plus-latest",
    api_key=os.getenv("QWEN_API_KEY"),
)  # 调用大模型

# 设置大模型的提示词
llm.prompt(ChatPrompter(instruction=robot_prompt, extra_keys=["context_str"]))
# 定义查询问题
query = "MIT OpenCourseWare是啥？"

# 创建查询改写器并改写查询
query_rewriter = llm.share(ChatPrompter(instruction=rewrite_prompt))
query = query_rewriter(query)
print(f"改写后的查询：\n{query}")

# 使用检索器检索相关文档
doc_node_list = retriever(query=query)

# 将查询和召回节点中的内容组成dict，作为大模型的输入
# 构造大模型输入参数，包括查询和上下文
res = llm(
    {
        "query": query,
        "context_str": "".join([node.get_content() for node in doc_node_list]),
    }
)

# 输出大模型的回答
print("\n回答: ", res)

### 子问题查询

将主问题拆解为多个子问题，有助于系统从不同角度理解问题并生成更全面的答案。**每个子问题单独检索，最后合并答案**。适用于**抽象或复合型查询**，但需注意处理性能开销。我们配合前面的查询扩写进行优化，上代码：

In [ ]:
# 子问题查询
# 导入必要的库
import lazyllm
from lazyllm import Document, ChatPrompter, Retriever

# 查询改写提示词
# 用于将用户的查询改写得更加清晰，不回答问题
rewrite_prompt = "你是一个查询改写助手，将用户的查询改写的更加清晰。\
                注意，你不需要对问题进行回答，只需要对原始问题进行改写。\
                下面是一个简单的例子：\
                输入：RAG\
                输出：为我介绍下RAG。\
                    \
                用户输入为："

# 子问题拆分提示词
# 用于将主问题拆解为多个子问题，有助于系统从不同角度理解问题
sub_query_prompt = "你是一个查询重写助手，如果用户查询较为抽象或笼统，将其分解为多个角度的具体问题。\
                  注意，你不需要进行回答，只需要根据问题的字面进行子问题拆分，输出不要超过3条。\
                  下面是一个简单的例子：\
                  输入：RAG是什么？\
                  输出：RAG的定义是什么？\
                       RAG是什么领域内的名词？\
                       RAG有什么特点？"

# AI问答助手提示词
# 用于根据上下文和问题提供答案
robot_prompt = "你是一个友好的 AI 问答助手，你需要根据给定的上下文和问题提供答案。\
                根据以下资料回答问题：\
                {context_str} \n"

# 加载文档库，定义检索器和在线大模型
# 创建文档对象，指定数据集路径
documents = Document(dataset_path="./data_kb")  # 请在 dataset_path 传入数据集绝对路径
# 创建检索器，指定文档、分组名、相似度算法和返回结果数量
retriever = Retriever(
    doc=documents, group_name="CoarseChunk", similarity="bm25_chinese", topk=3
)  # 定义检索组件
# 创建在线大模型对象，指定来源、模型和API密钥
llm = lazyllm.OnlineChatModule(
    source="qwen",
    model="qwen-plus-latest",
    api_key=os.getenv("QWEN_API_KEY"),
)  # 调用大模型

# 设置大模型的提示词
llm.prompt(ChatPrompter(instruction=robot_prompt, extra_keys=["context_str"]))
# 定义查询问题
query = "MIT OpenCourseWare是啥？"

# 创建查询改写器并改写查询
query_rewriter = llm.share(ChatPrompter(instruction=rewrite_prompt))
query = query_rewriter(query)
print(f"改写后的查询：\n{query}")

# 创建子问题生成器
sub_query_generator = llm.share(ChatPrompter(instruction=sub_query_prompt))
sub_queries_str = sub_query_generator(query)
sub_queries = [q.strip() for q in sub_queries_str.split('\n') if q.strip()]
print(f"拆分的子问题：\n{sub_queries}")

# 对每个子问题进行检索并合并结果
all_context = []
for sub_query in sub_queries:
    print(f"\n正在处理子问题: {sub_query}")
    # 使用检索器检索与子问题相关的文档
    doc_node_list = retriever(query=sub_query)
    # 将检索到的文档内容添加到上下文列表中
    for node in doc_node_list:
        all_context.append(node.get_content())

# 去重并合并上下文
# 使用set去除重复内容，然后合并为一个字符串
unique_context = list(set(all_context))
context_str = "".join(unique_context)
print(f"\n合并后的上下文长度: {len(context_str)}")

# 将查询和合并后的上下文作为大模型的输入
# 构造输入字典，包含查询和上下文
res = llm(
    {
        "query": query,
        "context_str": context_str,
    }
)

# 输出大模型的回答
print("\n回答: ", res)

这次的输出会多一段处理的子问题以及合并之后的上下文长度：

```bash
正在处理子问题: MIT OpenCourseWare 由哪个机构推出？

正在处理子问题: MIT OpenCourseWare 提供哪些类型的课程资源？

合并后的上下文长度: 3475
```

### 多步骤查询

将**复杂问题拆解为连续的多个步骤**，每一步依赖上一步的答案推进。**适合需要多轮推理的场景**，能有效提升对复杂问题的处理能力。我们配合前面的查询扩写进行优化，上代码：

In [ ]:
# 分步骤查询
# 导入必要的库
import lazyllm
import os
from lazyllm import Document, ChatPrompter, Retriever

# prompt设计
# 查询改写提示词，用于将用户的查询改写得更加清晰，不回答问题
rewrite_prompt = "你是一个查询改写助手，将用户的查询改写的更加清晰。\
                注意，你不需要对问题进行回答，只需要对原始问题进行改写。\
                下面是一个简单的例子：\
                输入：RAG\
                输出：为我介绍下RAG。\
                    \
                用户输入为："

# 判别提示词，用于判断某个回答能否解决对应的问题
judge_prompt = "你是一个判别助手，用于判断某个回答能否解决对应的问题。如果回答可以解决问题，则输出True，否则输出False。\
                注意，你的输出只能是True或者False。不要带有任何其他输出。\
                当前回答为{context_str} \n"

# AI问答助手提示词，用于根据上下文和问题提供答案
robot_prompt = "你是一个友好的 AI 问答助手，你需要根据给定的上下文和问题提供答案。\
                根据以下资料回答问题：\
                {context_str} \n"

# 加载文档库，定义检索器和在线大模型
# 创建文档对象，指定数据集路径
documents = Document(dataset_path="./data_kb")  # 请在 dataset_path 传入数据集绝对路径
# 创建检索器，指定文档、分组名、相似度算法和返回结果数量
retriever = Retriever(
    doc=documents, group_name="CoarseChunk", similarity="bm25_chinese", topk=3
)  # 定义检索组件
# 创建在线大模型对象，指定来源、模型和API密钥
llm = lazyllm.OnlineChatModule(
    source="qwen",
    model="qwen-plus-latest",
    api_key=os.getenv("QWEN_API_KEY"),
)  # 调用大模型

# 创建查询改写器
rewrite_robot = llm.share(ChatPrompter(instruction=rewrite_prompt))

# 创建AI问答助手
robot = llm.share(ChatPrompter(instruction=robot_prompt, extra_keys=["context_str"]))

# 创建判别助手
judge_robot = llm.share(ChatPrompter(instruction=judge_prompt, extra_keys=["context_str"]))

# 定义查询问题
query = "MIT OpenCourseWare是啥？"

# 分步骤查询逻辑
LLM_JUDGE = False
while LLM_JUDGE is not True:
    # 执行查询重写
    query_rewrite = rewrite_robot(query)
    print(f"\n重写的查询：{query_rewrite}")

    # 获取重写后的查询结果
    doc_node_list = retriever(query_rewrite)
    # 使用AI问答助手生成回答
    res = robot({"query": query_rewrite, "context_str": "\n".join([node.get_content() for node in doc_node_list])})

    # 判断当前回复是否能满足query要求
    LLM_JUDGE = bool(judge_robot({"query": query, "context_str": res}))
    print(f"\nLLM判断结果：{LLM_JUDGE}")

# 打印最终结果
print("\n最终回复: ", res)

这次的输出会多一段每一步骤的回复能否满足查询的要求，由`LLM_JUDGE`这个变量决定：

```bash
LLM判断结果：True
```